In [1]:
%matplotlib inline
import os
import cv2
import struct
import numpy as np
from matplotlib import pyplot as plt

In [2]:
def get_images_recursively(directory):
    directory = os.path.normpath(directory)
    image_id = 0
    res = dict()

    sorted_list = sorted(os.listdir(directory))

    i = 0
    for folder in sorted_list:
        if os.path.isdir(os.path.join(directory, folder)):
            for image in sorted(os.listdir(os.path.join(directory, folder))):
                res[image_id] = os.path.join(directory, folder, image)
                image_id += 1
        i += 1
        print(str(i) + "/" + str(len(sorted_list)), end="\r")
    return res

In [3]:
images = get_images_recursively("C:\\Users\\Tom\\Desktop\\images-4fps-selected\\images")

In [5]:
with open("C:\\Users\\Tom\\Desktop\\CT24.edges", "wb") as f:
    f.write(struct.pack("<I", len(images)))
    f.write(struct.pack("<I", 120))
    f.write(struct.pack("<I", 90))
    
    i = 0
    for image_id in sorted(images):
        image = images[image_id]
        
        f.write(struct.pack("<I", image_id))

        img = cv2.imread(image)
        #img = cv2.GaussianBlur(img, (15,15), 0)
        img = cv2.resize(img, (120,90))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 50, 300)
        
        
#         lines = cv2.HoughLines(edges, 2, np.pi/90, 60)
#         if lines is not None:
#             lines = [line[0] for line in lines]
#             for dist, angle in lines:
#                 a = np.cos(angle)
#                 b = np.sin(angle)
#                 x0 = a*dist
#                 y0 = b*dist
#                 x1 = int(x0 + 1000*(-b))
#                 y1 = int(y0 + 1000*a)
#                 x2 = int(x0 - 1000*(-b))
#                 y2 = int(y0 - 1000*a)
#                 cv2.line(img, (x1,y1), (x2,y2), (255,0,0), 1)

#         lines = cv2.HoughLinesP(edges, 2, np.pi/90, 1, 20, 5)
#         if lines is not None:
#             lines = [line[0] for line in lines]
#             for x1,y1,x2,y2 in lines:
#                 cv2.line(img, (x1,y1), (x2,y2), (255,0,0), 1)

#         im2, contours, hierarchy = cv2.findContours(edges,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
#         img = cv2.drawContours(img, contours, -1, (255,0,0), 1)

        ret, markers = cv2.connectedComponents(edges)
            
        data = struct.pack("=" + "B" * (120*90), *np.ndarray.flatten(markers))
        f.write(data)

#         plt.imshow(markers, cmap='gray')
#         plt.axis('off')
#         plt.show()
        i += 1
        print(str(i) + "/" + str(len(images)), end="\r")